In [1]:
import gc
import time
import pandas as pd
import numpy as np
import seaborn as sns
import random
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.model_selection import train_test_split, GridSearchCV
import scikitplot as skplt
from dateutil.relativedelta import *
from scipy.stats import bernoulli
from contextlib import contextmanager
from datetime import timedelta
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, precision_score
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from lifetimes.utils import summary_data_from_transaction_data

In [2]:
@contextmanager
def timer(name):
    time_before_func_call = time.time()
    yield
    time_after_func_call = time.time()
    print('{} - done in {:.0f}s'.format(name, time_after_func_call - time_before_func_call))

In [3]:
def rename_columns(invoices: pd.DataFrame) -> pd.DataFrame:
    invoices = invoices.copy()
    column_names = list(invoices.columns)
    if 'Customer ID' in column_names:
        invoices = invoices.rename(columns={'Customer ID': 'CustomerID'})
    if 'Invoice' in column_names:
        invoices = invoices.rename(columns={'Invoice': 'InvoiceNo'})
    if 'Price' in column_names:
        invoices = invoices.rename(columns={'Price': 'UnitPrice'})

    return invoices


def concat_invoice_dataframes(invoices1: pd.DataFrame, invoices2: pd.DataFrame) -> pd.DataFrame:
    invoices1, invoices2 = invoices1.copy(), invoices2.copy()
    end_of_overlap_period = '2010-12-10'  # Specific to the two UCI online retail datasets

    if invoices1.InvoiceDate.max() > invoices2.InvoiceDate.max():
        invoices1 = invoices1[invoices1.InvoiceDate > end_of_overlap_period]
        invoices = pd.concat([invoices2, invoices1])
    else:
        invoices2 = invoices2[invoices2.InvoiceDate > end_of_overlap_period]
        invoices = pd.concat([invoices1, invoices2])

    return invoices


def add_revenue_column(invoices: pd.DataFrame) -> pd.DataFrame:
    invoices = invoices.copy()
    if "Revenue" not in list(invoices.columns):
        invoices['Revenue'] = invoices['UnitPrice'] * invoices['Quantity']

    return invoices


def drop_test_invoices(invoices: pd.DataFrame) -> pd.DataFrame:
    invoices = invoices.copy()
    test_invoice_indexs = invoices[invoices['StockCode'].str.contains('TEST', case=False, na=False)].index
    invoices = invoices.drop(index=test_invoice_indexs)

    return invoices


def drop_cancellation_invoices(invoices: pd.DataFrame) -> pd.DataFrame:
    invoices = invoices.copy()
    cancellation_invoice_indexs = invoices[invoices["InvoiceNo"].str.contains('c', na=False, case=False)].index
    invoices = invoices.drop(index=cancellation_invoice_indexs)

    return invoices


def drop_return_invoices(invoices: pd.DataFrame) -> pd.DataFrame:
    invoices = invoices.copy()
    invoices = invoices[invoices['UnitPrice'] > 0].copy()

    return invoices


def drop_non_numeric_invoice_numbers(invoices: pd.DataFrame) -> pd.DataFrame:
    invoices = invoices.copy()
    invoices = invoices[pd.to_numeric(invoices['InvoiceNo'], errors='coerce').notna()]
    return invoices


def clean_stock_codes(invoices):
    invoices_copy = invoices.copy()
    invoices_copy.drop(index=invoices_copy[invoices_copy.StockCode == 'C2'].index, inplace=True)
    invoices_copy.drop(index=invoices_copy[invoices_copy.StockCode == 'C3'].index, inplace=True)
    invoices_copy['StockCode'] = invoices_copy['StockCode'].str.replace("^\D+$", "Not an Item")
    invoices_copy["StockCode"] = invoices_copy["StockCode"].str.replace("gift.*", "Not an Item")
    invoices_copy.drop(index=invoices_copy[invoices_copy.StockCode == 'Not an Item'].index, inplace=True)
    invoices_copy['StockCode'] = invoices_copy['StockCode'].str.replace("\D+$", "")

    return invoices_copy


In [4]:
with timer("Read invoice data"):
    invoices_2011 = pd.read_excel("data/retail/Online_Retail.xlsx")
    invoices_2010 = pd.read_excel("data/retail/online_retail_II.xlsx")
    
with timer("Preprocess invoice dataframes"):
    invoices_2010 = rename_columns(invoices_2010)
    invoices = concat_invoice_dataframes(invoices_2010, invoices_2011)
    invoices = add_revenue_column(invoices)
    invoices['StockCode'] = invoices['StockCode'].astype(str)
    invoices = drop_test_invoices(invoices)
    invoices = drop_return_invoices(invoices)
    invoices = drop_non_numeric_invoice_numbers(invoices)
    invoices = clean_stock_codes(invoices)

Read invoice data - done in 124s
Preprocess invoice dataframes - done in 5s


<br>
<br>

In [5]:
invoices.to_csv("invoices_2009_2011_no_returns.csv", index=False)